### Import Required Libraries and Set Up Environment Variables

In [55]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

In [3]:
nyt_api_key

'zGYmRzfPDVolX9IbkCvRvz8NEM270zuu'

In [4]:
tmdb_api_key

'e28b22ee3ea6bf5684e0d279c18d49b8'

### Access the New York Times API

In [7]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
full_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}"
full_url

'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=zGYmRzfPDVolX9IbkCvRvz8NEM270zuu'

In [8]:
requests.get(full_url).json()

{'status': 'OK',
 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.',
 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html',
    'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.',
    'source': 'The New York Times',
    'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick',
     'kicker': None,
     'content_kicker': None,
     'print_headline': 'The Attachment Diaries',
     'name': None,
     'seo': None,
     'sub': None},
    'keywords': [{'name': 'subject',
      'value': 'Movies',
      'rank': 1,
      'major': 'N'},
     {'name': 'creative_works',
      'value': 'The Attachment Diaries (Movie)',
      'rank': 2,
      'major': 'N'},
     {'name': 'persons',
      'value': 'Diment, Valentin Javier',
      'rank': 3,
      'major': 'N'}],
    'pub_date': '2023-05-25T11:00:03+0000',
    'byline': {'orig

In [35]:
# Create an empty list to store the reviews
reviews = []

# loop through pages 0-19
for i in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    paginated_url = f"{full_url}&page={i}"
    
    # Make a "GET" request and retrieve the JSON
    json_response = requests.get(paginated_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    for doc in json_response['response']['docs']:
        # loop through the reviews["response"]["docs"] and append each review to the list
        reviews.append(doc)
        # Print the page that was just retrieved
    if len(json_response['response']['docs']) > 0:
        print(f'Checked page {i}')
    else:
        print(f'Page {i} - no results')
        
        # Print the page number that had no results then break from the loop
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [36]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [37]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_df = pd.json_normalize(reviews)
nyt_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [38]:
nyt_df['headline.main'].str.extract(r'\u2018(.*?)(?=\u2019 Review|$)')

,0
0,The Attachment Diaries
1,What’s Love Got to Do With It?’ Probably a Lot
2,You Can Live Forever
3,A Tourist’s Guide to Love
4,Other People’s Children
...,...
195,The Other Half’
196,The Ottoman Lieutenant’
197,Love & Taxes’
198,"Everybody Loves Somebody,’ a Rom-Com With Bite"


In [46]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
nyt_df['title'] = nyt_df['headline.main'].str.extract(r'\u2018(.*?)(?=\u2019\s|$Review)')
nyt_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,NaN
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,NaN
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,NaN
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"


In [47]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_df['keywords'] = nyt_df['keywords'].apply(extract_keywords)
nyt_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,NaN
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,NaN
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,NaN
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"


In [41]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = nyt_df['title'].to_list()


### Access The Movie Database API

In [42]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [43]:
requests.get(f"{url}{title_list[0]}{tmdb_key_string}").json()

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': None,
   'genre_ids': [18, 9648, 53, 27],
   'id': 743040,
   'original_language': 'es',
   'original_title': 'El apego',
   'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.',
   'popularity': 1.4,
   'poster_path': '/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg',
   'release_date': '2021-10-07',
   'title': 'The Attachment Diaries',
   'video': False,
   'vote_average': 4.0,
   'vote_count': 2}],
 'total_pages': 1,
 'total_results': 1}

In [50]:
id_url = "https://api.themoviedb.org/3/movie/"

In [51]:
requests.get(f"{id_url}743040?api_key={tmdb_api_key}").json()

{'adult': False,
 'backdrop_path': None,
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 18, 'name': 'Drama'},
  {'id': 9648, 'name': 'Mystery'},
  {'id': 53, 'name': 'Thriller'},
  {'id': 27, 'name': 'Horror'}],
 'homepage': '',
 'id': 743040,
 'imdb_id': 'tt11359158',
 'original_language': 'es',
 'original_title': 'El apego',
 'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.',
 'popularity': 1.792,
 'poster_path': '/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg',
 'production_companies': [],
 'production_countries': [{'iso_3166_1': 'AR', 'name': 'Argentina'}],
 'release_date': '2021-10-07',
 'revenue': 0,
 'runtime': 102,
 'spoken_lang

In [60]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
num_requests = 0

# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    if num_requests > 49:
        time.sleep(12)
        num_requests = 0
    
    # Add 1 to the request counter
    num_requests += 1
    # Perform a "GET" request for The Movie Database
    json_response = requests.get(f'{url}{title}{tmdb_key_string}').json()
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        id = json_response['results'][0]['id']
        
        # Make a request for a the full movie details
        id_response = requests.get(f"{id_url}{id}?api_key={tmdb_api_key}").json()
        
        # Execute "GET" request with url
        # Extract the genre names into a list
        genre_list = [genre['name'] for genre in id_response['genres']]

        # Extract the spoken_languages' English name into a list
        language_list = [language['english_name'] for language in id_response['spoken_languages']]


        # Extract the production_countries' name into a list
        country_list = [country['name'] for country in id_response['production_countries']]


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            'title': title,
            'original_title': id_response['original_title'],
            'budget': id_response['budget'],
            'language': id_response['original_language'],
            'homepage': id_response['homepage'],
            'overview': id_response['overview'],
            'popularity': id_response['popularity'],
            'runtime': id_response['runtime'],
            'revenue': id_response['revenue'],
            'release_date': id_response['release_date'],
            'vote_average': id_response['vote_average'],
            'vote_count': id_response['vote_count'],
            'genre': genre_list,
            'spoken_languages': language_list,
            'production_countries': country_list
        })
    
        # Print out the title that was found
        print(f'Found {title}')
    except:
        print(f'{title} not found')


Found The Attachment Diaries
Found What’s Love Got to Do With It?
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
Found Cyrano
Found Th

In [61]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
json.dumps(tmdb_movies_list, indent=4)

'[\n    {\n        "title": "The Attachment Diaries",\n        "original_title": "El apego",\n        "budget": 0,\n        "language": "es",\n        "homepage": "",\n        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",\n        "popularity": 1.792,\n        "runtime": 102,\n        "revenue": 0,\n        "release_date": "2021-10-07",\n        "vote_average": 4.0,\n        "vote_count": 2,\n        "genre": [\n            "Drama",\n            "Mystery",\n            "Thriller",\n            "Horror"\n        ],\n        "spoken_languages": [\n            "Spanish"\n        ],\n        "production_countries": [\n            "Argenti

In [63]:
# Convert the results to a DataFrame

tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.792,102,0,2021-10-07,4.000,2,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,en,,Two childhood friends now in their thirties mu...,19.007,109,10898395,2023-01-26,6.100,153,"[Romance, Comedy]","[English, Urdu, Portuguese]","[France, United Kingdom]"
2,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",6.331,96,15055,2023-03-24,6.750,28,"[Drama, Romance]","[English, French]",[Canada]
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",25.041,96,0,2023-04-21,6.365,144,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",13.056,104,84178,2022-09-21,6.914,168,"[Drama, Comedy]","[English, French]",[France]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,NaN,Nan,10000,ku,,Sahim Omar Kalifa's graduating Film. A Kurdish...,0.658,30,0,2008-09-08,5.300,3,"[Drama, Crime]","[Dutch, Kurdish]","[Belgium, Iraq]"
195,NaN,Nan,10000,ku,,Sahim Omar Kalifa's graduating Film. A Kurdish...,0.658,30,0,2008-09-08,5.300,3,"[Drama, Crime]","[Dutch, Kurdish]","[Belgium, Iraq]"
196,NaN,Nan,10000,ku,,Sahim Omar Kalifa's graduating Film. A Kurdish...,0.658,30,0,2008-09-08,5.300,3,"[Drama, Crime]","[Dutch, Kurdish]","[Belgium, Iraq]"
197,"Everybody Loves Somebody,",Everybody Loves Somebody,2000000,en,,"On the surface, Clara Barron seems to have it ...",9.831,102,0,2017-02-10,6.566,158,"[Romance, Comedy]","[English, Spanish]",[Mexico]


### Merge and Clean the Data for Export

In [80]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = nyt_df.merge(tmdb_df, on="title")
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.792,102,0,2021-10-07,4.000,2,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,Two childhood friends now in their thirties mu...,19.007,109,10898395,2023-01-26,6.100,153,"[Romance, Comedy]","[English, Urdu, Portuguese]","[France, United Kingdom]"
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",6.331,96,15055,2023-03-24,6.750,28,"[Drama, Romance]","[English, French]",[Canada]
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",25.041,96,0,2023-04-21,6.365,144,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",13.056,104,84178,2022-09-21,6.914,168,"[Drama, Comedy]","[English, French]",[France]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,https://www.nytimes.com/2017/04/20/movies/the-...,"Inspired by a true story, “The Happiest Day in...",The New York Times,subject: Movies;creative_works: The Happiest D...,2017-04-20T20:45:54+0000,824,Review: In ‘The Happiest Day in the Life of Ol...,None,None,Love and Life on the Ropes,...,Summer 1962 and Olli Mäki has a shot at the wo...,7.969,95,0,2016-09-02,6.663,80,"[Drama, Romance]","[English, Finnish]","[Germany, Finland, Sweden]"
407,https://www.nytimes.com/2017/04/20/movies/the-...,"Oscar Isaac, Christian Bale and Charlotte Le B...",The New York Times,subject: Movies;creative_works: The Promise (M...,2017-04-20T16:49:46+0000,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",...,Set during the last days of the Ottoman Empire...,11.236,133,12448676,2016-12-02,6.940,435,"[History, Romance, Drama]","[Armenian, German, English, French]","[Spain, United States of America]"
408,https://www.nytimes.com/2017/03/14/movies/fran...,François Ozon’s adaptation of Ernst Lubitsch’s...,The New York Times,"creative_works: Frantz (Movie);persons: Niney,...",2017-03-14T21:14:24+0000,708,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",None,None,The Wounds of War and Love,...,"In the aftermath of WWI, a young German who gr...",11.528,113,7478354,2016-09-07,7.377,654,"[History, Drama, Romance]","[German, French]","[France, Germany]"
409,https://www.nytimes.com/2017/02/16/movies/ever...,

In [81]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
fix_list = ['genre', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
remove_characters_list = ['[', ']', "'"]

def remove_characters(value):
    for char in remove_characters_list:
        value = value.replace(char, "")
    return value

# Loop through the list of columns to fix
for col in fix_list:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)
    
    # Loop through characters to remove
    merged_df[col] = merged_df[col].apply(remove_characters)

# Display the fixed DataFrame
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.792,102,0,2021-10-07,4.000,2,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,Two childhood friends now in their thirties mu...,19.007,109,10898395,2023-01-26,6.100,153,"Romance, Comedy","English, Urdu, Portuguese","France, United Kingdom"
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",6.331,96,15055,2023-03-24,6.750,28,"Drama, Romance","English, French",Canada
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",25.041,96,0,2023-04-21,6.365,144,"Romance, Comedy","English, Vietnamese",United States of America
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",13.056,104,84178,2022-09-21,6.914,168,"Drama, Comedy","English, French",France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,https://www.nytimes.com/2017/04/20/movies/the-...,"Inspired by a true story, “The Happiest Day in...",The New York Times,subject: Movies;creative_works: The Happiest D...,2017-04-20T20:45:54+0000,824,Review: In ‘The Happiest Day in the Life of Ol...,None,None,Love and Life on the Ropes,...,Summer 1962 and Olli Mäki has a shot at the wo...,7.969,95,0,2016-09-02,6.663,80,"Drama, Romance","English, Finnish","Germany, Finland, Sweden"
407,https://www.nytimes.com/2017/04/20/movies/the-...,"Oscar Isaac, Christian Bale and Charlotte Le B...",The New York Times,subject: Movies;creative_works: The Promise (M...,2017-04-20T16:49:46+0000,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",...,Set during the last days of the Ottoman Empire...,11.236,133,12448676,2016-12-02,6.940,435,"History, Romance, Drama","Armenian, German, English, French","Spain, United States of America"
408,https://www.nytimes.com/2017/03/14/movies/fran...,François Ozon’s adaptation of Ernst Lubitsch’s...,The New York Times,"creative_works: Frantz (Movie);persons: Niney,...",2017-03-14T21:14:24+0000,708,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",None,None,The Wounds of War and Love,...,"In the aftermath of WWI, a young German who gr...",11.528,113,7478354,2016-09-07,7.377,654,"History, Drama, Romance","German, French","France, Germany"
409,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between..

In [82]:
# Drop "byline.person" column
merged_df.drop(columns="byline.person", inplace=True)
merged_df.columns

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.organization', 'title',
       'original_title', 'budget', 'language', 'homepage', 'overview',
       'popularity', 'runtime', 'revenue', 'release_date', 'vote_average',
       'vote_count', 'genre', 'spoken_languages', 'production_countries'],
      dtype='object')

In [83]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index()

In [84]:
merged_df

,index,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre,spoken_languages,production_countries
0,0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,...,"Argentina, 1970s. A desperate young woman goes...",1.792,102,0,2021-10-07,4.000,2,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,...,Two childhood friends now in their thirties mu...,19.007,109,10898395,2023-01-26,6.100,153,"Romance, Comedy","English, Urdu, Portuguese","France, United Kingdom"
2,2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,...,"When Jaime, a gay teenager, is sent to live in...",6.331,96,15055,2023-03-24,6.750,28,"Drama, Romance","English, French",Canada
3,3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,...,"After an unexpected break up, a travel executi...",25.041,96,0,2023-04-21,6.365,144,"Romance, Comedy","English, Vietnamese",United States of America
4,4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,...,"Rachel loves her life, her students, her frien...",13.056,104,84178,2022-09-21,6.914,168,"Drama, Comedy","English, French",France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,406,https://www.nytimes.com/2017/04/20/movies/the-...,"Inspired by a true story, “The Happiest Day in...",The New York Times,subject: Movies;creative_works: The Happiest D...,2017-04-20T20:45:54+0000,824,Review: In ‘The Happiest Day in the Life of Ol...,None,None,...,Summer 1962 and Olli Mäki has a shot at the wo...,7.969,95,0,2016-09-02,6.663,80,"Drama, Romance","English, Finnish","Germany, Finland, Sweden"
195,407,https://www.nytimes.com/2017/04/20/movies/the-...,"Oscar Isaac, Christian Bale and Charlotte Le B...",The New York Times,subject: Movies;creative_works: The Promise (M...,2017-04-20T16:49:46+0000,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,...,Set during the last days of the Ottoman Empire...,11.236,133,12448676,2016-12-02,6.940,435,"History, Romance, Drama","Armenian, German, English, French","Spain, United States of America"
196,408,https://www.nytimes.com/2017/03/14/movies/fran...,François Ozon’s adaptation of Ernst Lubitsch’s...,The New York Times,"creative_works: Frantz (Movie);persons: Niney,...",2017-03-14T21:14:24+0000,708,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",None,None,...,"In the aftermath of WWI, a young German who gr...",11.528,113,7478354,2016-09-07,7.377,654,"History, Drama, Romance","German, French","France, Germany"
197,409,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,...,"On the surface, Clara Barron seems to have it ...",9.831,10

In [85]:
# Export data to CSV without the index
merged_df.to_csv('movies.csv', index=False)